In [1]:
import pandas as pd
import pyodbc


In [2]:
DB = {'servername': 'LAPTOP-105ECA33\SQLEXPRESS', 
    'database': 'UnitedOutdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor  

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\antho\AppData\Local\Temp\ipykernel_4808\944234878.py:1: SyntaxWarning: invalid escape sequence '\S'
  DB = {'servername': 'LAPTOP-105ECA33\SQLEXPRESS',


In [3]:

Adventure = {
    'servername' : 'LAPTOP-105ECA33\SQLEXPRESS',
    'database' : 'AdventureWorks2019'
}


Northwind = {
    'servername' : 'LAPTOP-105ECA33\SQLEXPRESS',
    'database' : 'Northwind'
}

access_db_path = r'C:\HBO ICT\DEDS jaar 2\aenc.accdb'

#Connect to AdventureWorks
Adventure_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Adventure['servername']};DATABASE={Adventure['database']};Trusted_Connection=yes;")
Adventure_cursor = Adventure_conn.cursor()

#Connect to Northwind
Northwind_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Northwind['servername']};DATABASE={Northwind['database']};Trusted_Connection=yes;")
Northwind_cursor = Northwind_conn.cursor()

#Connect to AenC
AenC_conn = pyodbc.connect(f"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};")



<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\S'
C:\Users\antho\AppData\Local\Temp\ipykernel_4808\1537168005.py:2: SyntaxWarning: invalid escape sequence '\S'
  'servername' : 'LAPTOP-105ECA33\SQLEXPRESS',
C:\Users\antho\AppData\Local\Temp\ipykernel_4808\1537168005.py:8: SyntaxWarning: invalid escape sequence '\S'
  'servername' : 'LAPTOP-105ECA33\SQLEXPRESS',


In [4]:
AdventureWorks_queries = {
    'Employee' : 'SELECT * FROM HumanResources.Employee',
    
}

Northwind_queries = {
    'Employee' : 'SELECT * FROM Employees',
    
}

AenC_queries = {
    'Employee' : 'SELECT * FROM Employee',
    
}

dataframes = {}

# Lees elke tabel in een DataFrame

for table_name, query in AdventureWorks_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)

for table_name, query in Northwind_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)

for table_name, query in AenC_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, AenC_conn)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df


C:\Users\antho\AppData\Local\Temp\ipykernel_4808\3154358536.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)
C:\Users\antho\AppData\Local\Temp\ipykernel_4808\3154358536.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)
C:\Users\antho\AppData\Local\Temp\ipykernel_4808\3154358536.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, AenC_con

In [ ]:
#eerst horizontaal samenvoegen per database:
AdventureWorks_Order = pd.merge(SalesOrderHeader, SalesOrderDetail, on = 'SalesOrderID')
Northwind_Order = pd.merge(Orders, OrderDetails, on = 'OrderID')
AenC_Order = pd.merge(sales_order, sales_order_item, on = 'id')

In [ ]:
AdventureWorks_Order.rename(columns={
    'SalesOrderID': 'OrderID',
    'SalesOrderDetailID': 'OrderDetailID',
    'OrderQty': 'Quantity',
    'UnitPriceDiscount': 'Discount',
    'Order_Date': 'OrderDate',
    'SalesPersonID' : 'EmployeeID'
}, inplace=True)

# change shipvia to string 
#Northwind_Order['ShipVia'] = Northwind_Order['ShipVia'].astype(str)
Northwind_Order.rename(columns={
    'ShipVia': 'ShipMethodID'
}, inplace=True)

AenC_Order.rename(columns={
    'id': 'OrderID',
    'item_id': 'OrderDetailID',
    'quantity': 'Quantity',
    'order_date': 'OrderDate',
    'cust_id': 'CustomerID',
    'sales_rep': 'EmployeeID',
    'prod_id': 'ProductID'
}, inplace=True)

order_order_ids = set(AdventureWorks_Order['OrderID'])
order_details_order_ids = set(Northwind_Order['OrderID'])
Order_AenC_ids = set(AenC_Order['OrderID'])

# Zoek conflicten tussen AdventureWorks en Northwind
conflicting_order_ids_1 = order_order_ids.intersection(order_details_order_ids)

# Zoek conflicten tussen AdventureWorks en AenC
conflicting_order_ids_2 = order_order_ids.intersection(Order_AenC_ids)

# Zoek conflicten tussen Northwind en AenC
conflicting_order_ids_3 = order_details_order_ids.intersection(Order_AenC_ids)

# Combineer alle conflicterende ID's
all_conflicting_order_ids = conflicting_order_ids_1.union(conflicting_order_ids_2).union(conflicting_order_ids_3)

if all_conflicting_order_ids:
    print("Conflicting Order IDs found:", all_conflicting_order_ids)
else:
    print("No conflicts found in Order IDs.")

# Display the conflicting Order IDs (if any)
conflicts_df = pd.DataFrame(list(all_conflicting_order_ids), columns=['Conflicting Order IDs'])
print(conflicts_df)

No conflicts found in Order IDs.
Empty DataFrame
Columns: [Conflicting Order IDs]
Index: []


In [ ]:
# Adding a prefix to EmployeeIDs for each source DataFrame
AdventureWorks_Order['EmployeeID'] = AdventureWorks_Order['EmployeeID'].apply(lambda x: f'AW_{x}')
Northwind_Order['EmployeeID'] = Northwind_Order['EmployeeID'].apply(lambda x: f'NW_{x}')
AenC_Order['EmployeeID'] = AenC_Order['EmployeeID'].apply(lambda x: f'AC_{x}')

#prefix for SHIPMETHOD

AdventureWorks_Order['ShipMethodID'] = AdventureWorks_Order['ShipMethodID'].apply(lambda x: f'AW_{x}')
Northwind_Order['ShipMethodID'] = Northwind_Order['ShipMethodID'].apply(lambda x: f'NW_{x}')
#AenC_Order['ShipMethodID'] = AenC_Order['ShipMethodID'].apply(lambda x: f'AC_{x}')

#prefix for ProductID

AdventureWorks_Order['ProductID'] = AdventureWorks_Order['ProductID'].apply(lambda x: f'AW_{x}')
Northwind_Order['ProductID'] = Northwind_Order['ProductID'].apply(lambda x: f'NW_{x}')
AenC_Order['ProductID'] = AenC_Order['ProductID'].apply(lambda x: f'AC_{x}')

#prefix for CustomerID

AdventureWorks_Order['CustomerID'] = AdventureWorks_Order['CustomerID'].apply(lambda x: f'AW_{x}')
#Northwind_Order['CustomerID'] = Northwind_Order['CustomerID'].apply(lambda x: f'NW_{x}')
AenC_Order['CustomerID'] = AenC_Order['CustomerID'].apply(lambda x: f'AC_{x}')

# maar dan ook prefix zetten in bijbehorende dimensies en tabellen.
